In [1]:
import sys
print(sys.executable)


/opt/homebrew/Cellar/jupyterlab/4.4.6/libexec/bin/python


In [2]:
import sys
!{sys.executable} -m pip install spacy sentence-transformers scikit-learn unidecode pandas matplotlib
!{sys.executable} -m spacy download es_core_news_md


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.3/42.3 MB 3.7 MB/s  0:00:11m0:00:0100:01
✔ Download and installation successful
You can now load the package via spacy.load('es_core_news_md')


In [ ]:
import sys
!{sys.executable} -m pip install spacy sentence-transformers scikit-learn unidecode pandas matplotlib
!{sys.executable} -m spacy download es_core_news_md


  Using cached sentence_transformers-5.1.0-py3-none-any.whl.metadata (16 kB)
  Using cached scikit_learn-1.7.1-cp313-cp313-macosx_12_0_arm64.whl.metadata (11 kB)
  Using cached pandas-2.3.2-cp313-cp313-macosx_11_0_arm64.whl.metadata (91 kB)
  Using cached matplotlib-3.10.6-cp313-cp313-macosx_11_0_arm64.whl.metadata (11 kB)
  Using cached transformers-4.56.0-py3-none-any.whl.metadata (40 kB)
  Using cached torch-2.8.0-cp313-none-macosx_11_0_arm64.whl.metadata (30 kB)
  Using cached huggingface_hub-0.34.4-py3-none-any.whl.metadata (14 kB)
  Using cached pillow-11.3.0-cp313-cp313-macosx_11_0_arm64.whl.metadata (9.0 kB)
  Using cached filelock-3.19.1-py3-none-any.whl.metadata (2.1 kB)
  Using cached regex-2025.9.1-cp313-cp313-macosx_11_0_arm64.whl.metadata (40 kB)
  Using cached tokenizers-0.22.0-cp39-abi3-macosx_11_0_arm64.whl.metadata (6.8 kB)
  Using cached safetensors-0.6.2-cp38-abi3-macosx_11_0_arm64.whl.metadata (4.1 kB)
  Using cached fsspec-2025.7.0-py3-none-any.whl.metadata (12 kB

In [3]:
import spacy, re
from pathlib import Path

FILE_PATH = "../data/martes-interdisciplina-estudiantes-josefa-quezada.txt"

try:
    nlp = spacy.load("es_core_news_md")
except:
    nlp = spacy.load("es_core_news_sm")

text = Path(FILE_PATH).read_text(encoding="utf-8", errors="ignore")
text = re.sub(r"\s+", " ", text).strip()

doc = nlp(text)
sents = [s.text.strip() for s in doc.sents if s.text.strip()]

print(f"Oraciones detectadas: {len(sents)}")
print("Ejemplo:", sents[0][:200], "…")


Oraciones detectadas: 46
Ejemplo: DÍA 1 Bloque estudiantes - Interdisciplina JOSEFA QUEZADA Buenos días. …


In [5]:
# ================== CONFIG ==================
FILE_PATH = "../data/martes-interdisciplina-estudiantes-josefa-quezada.txt"

AXIS_TERMS = [
    "interdisciplina","interdisciplinar","interdisciplinario",
    "trabajo interdisciplinario","cruce de disciplinas","colaboración entre disciplinas"
]
# Léxico de diseño (puedes ampliarlo)
DESIGN_LEXICON = {
    "diseño","proyecto","prototipo","investigación","experiencia","narrativa","interfaz",
    "materialidad","tipografía","espacio","interacción","usuario","visual","sonoro",
    "exhibición","instalación","especulativo","metodología","iteración","proceso",
    "tecnología","multimedia","analogico","digital","sensorial","arquitectónico","inmersivo"
}

TOP_SENTENCES = 120             # más amplio para tener material
SIM_THRESHOLD_PERCENTILE = 55   # un poco más laxo
MIN_SNIPPET_WORDS = 2           # <<< NUEVO: mínimo 2 palabras
MAX_SNIPPET_WORDS = 8           # <<< NUEVO: máximo 8 palabras
MIN_CONCEPT_WORDS = 2           # <<< NUEVO: conceptos 2-3 palabras
MAX_CONCEPT_WORDS = 3

STOP_SNIPPETS = {
    "gracias","muchas gracias","ok","vale","sí","si","hola","buenas","buenos dias","buenas tardes","buenas noches",
    "este video","el video","video"
}

# ================== IMPORTS ==================
import re, json
from pathlib import Path
from collections import Counter, defaultdict
import numpy as np
import spacy
from unidecode import unidecode

# Cargar spaCy
try:
    nlp = spacy.load("es_core_news_md")
except Exception:
    nlp = spacy.load("es_core_news_sm")

# Embeddings (opcional)
use_st = True
try:
    from sentence_transformers import SentenceTransformer, util
    embedder = SentenceTransformer("sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2")
except Exception:
    print("⚠️ Sin Sentence-Transformers: usaré similitud de spaCy como respaldo.")
    use_st = False

# ================== CARGA & SEGMENTACIÓN ==================
text = Path(FILE_PATH).read_text(encoding="utf-8", errors="ignore")
text = re.sub(r"\s+", " ", text).strip()
doc = nlp(text)
sentences = [s.text.strip() for s in doc.sents if s.text.strip()]
if not sentences:
    raise ValueError("No se detectaron oraciones.")

# ================== SELECCIÓN DE ORACIONES CANDIDATAS ==================
axis_regex = re.compile(r"\binterdisciplin\w*|interdisciplina\w*\b", flags=re.IGNORECASE)

# 1) Semántica
if use_st:
    sent_emb = embedder.encode(sentences, convert_to_tensor=True, normalize_embeddings=True)
    axis_emb = embedder.encode(AXIS_TERMS, convert_to_tensor=True, normalize_embeddings=True)
    axis_vec = axis_emb.mean(dim=0)
    sims = util.cos_sim(sent_emb, axis_vec).cpu().numpy().reshape(-1)
else:
    axis_doc = nlp(" ".join(AXIS_TERMS))
    sims = np.array([nlp(s).similarity(axis_doc) for s in sentences], dtype="float32")

threshold = np.percentile(sims, SIM_THRESHOLD_PERCENTILE)
idx_sorted = np.argsort(-sims)
sem_idx = [i for i in idx_sorted if sims[i] >= threshold][:TOP_SENTENCES]

# 2) Keyword explícita
kw_idx = [i for i, s in enumerate(sentences) if axis_regex.search(s)]

# Unimos (priorizando más similares)
candidate_idx, seen = [], set()
for i in idx_sorted:
    if i in sem_idx or i in kw_idx:
        if i not in seen:
            candidate_idx.append(i); seen.add(i)
    if len(candidate_idx) >= TOP_SENTENCES:
        break
candidate_sents = [sentences[i] for i in candidate_idx]

# ================== UTILIDADES ==================
def normalize_token(t):
    t = re.sub(r"[^\wáéíóúñüÁÉÍÓÚÑÜ-]+", "", t.lower())
    return unidecode(t)

def contains_anchor(phrase_norm):
    toks = phrase_norm.split()
    # debe tener al menos una palabra ancla del eje o del léxico de diseño
    return any(t in DESIGN_LEXICON or re.search(r"^interdisciplin", t) for t in toks)

def trim_phrase(tokens, min_w=MIN_SNIPPET_WORDS, max_w=MAX_SNIPPET_WORDS):
    toks = [normalize_token(t) for t in tokens if normalize_token(t)]
    if len(toks) < min_w:
        return ""
    if len(toks) > max_w:
        toks = toks[:max_w]
    phrase = " ".join(toks)
    if phrase in STOP_SNIPPETS:
        return ""
    # descarta frases con solo 1 palabra distinta repetida
    if len(set(toks)) == 1 and len(toks) <= 3:
        return ""
    if not contains_anchor(phrase):
        return ""
    return phrase

# ================== EXTRACCIÓN DE FRAGMENTOS (2–8 palabras) ==================
snippets = []
seen_snips = set()

for s in candidate_sents:
    d = nlp(s)

    # A) Ventanas centradas en la keyword "interdiscipl..."
    toks = s.split()
    kw_positions = [i for i, t in enumerate(toks) if axis_regex.search(t)]
    for pos in kw_positions:
        radius = max(1, (MAX_SNIPPET_WORDS - 1)//2)
        left = max(0, pos - radius)
        right = min(len(toks), pos + radius + 1)
        phrase = trim_phrase(toks[left:right])
        if phrase and phrase not in seen_snips:
            snippets.append(phrase); seen_snips.add(phrase)

    # B) Noun chunks (compuestos nominales)
    for nc in d.noun_chunks:
        phrase = trim_phrase([t.text for t in nc])
        if phrase and phrase not in seen_snips:
            snippets.append(phrase); seen_snips.add(phrase)

    # C) Secuencias ADJ/NOUN/PROPN consecutivas (hasta 8 palabras)
    buf = []
    for t in d:
        if t.pos_ in ("ADJ","NOUN","PROPN") and not t.is_stop:
            buf.append(t.text)
        else:
            phrase = trim_phrase(buf)
            if phrase and phrase not in seen_snips:
                snippets.append(phrase); seen_snips.add(phrase)
            buf = []
    phrase = trim_phrase(buf)
    if phrase and phrase not in seen_snips:
        snippets.append(phrase); seen_snips.add(phrase)

# Backup: si quedaron muy pocos, toma inicios de oración con ancla
if len(snippets) < 20:
    for s in candidate_sents[:60]:
        toks = s.split()
        phrase = trim_phrase(toks[:MAX_SNIPPET_WORDS+2])  # un pelín más amplio
        if phrase and phrase not in seen_snips:
            snippets.append(phrase); seen_snips.add(phrase)

# ================== CONCEPTOS (2–3 palabras) ==================
def extract_concepts_from_snippets(snips, min_w=MIN_CONCEPT_WORDS, max_w=MAX_CONCEPT_WORDS):
    concepts = []
    for sn in snips:
        d = nlp(sn)
        buf = []
        for t in d:
            if t.pos_ in ("ADJ","NOUN","PROPN") and not t.is_stop and len(t.lemma_)>1:
                buf.append(unidecode(t.lemma_.lower()))
        if min_w <= len(buf) <= max_w:
            concepts.append(" ".join(buf[:max_w]))
        elif len(buf) > max_w:
            concepts.append(" ".join(buf[:max_w]))
    # Filtro: que contengan al menos una ancla del léxico, para evitar “proyecto” suelto
    concepts = [c for c in concepts if contains_anchor(c)]
    return Counter(concepts)

concept_freq = extract_concepts_from_snippets(snippets, MIN_CONCEPT_WORDS, MAX_CONCEPT_WORDS)
top_concepts = concept_freq.most_common(30)

# ================== CLUSTERS (opcionales) SOBRE FRAGMENTOS ==================
clusters_out = []
if use_st and snippets:
    from sklearn.cluster import KMeans
    X = embedder.encode(snippets, normalize_embeddings=True)
    k = 2 if len(snippets) < 30 else (3 if len(snippets) < 60 else 4)
    kmeans = KMeans(n_clusters=k, n_init=10, random_state=42)
    labels = kmeans.fit_predict(X)
    buckets = defaultdict(list)
    for lab, sn in zip(labels, snippets):
        buckets[int(lab)].append(sn)
    for c, items in buckets.items():
        sims_sn = util.cos_sim(embedder.encode(items, convert_to_tensor=True, normalize_embeddings=True),
                               axis_vec).cpu().numpy().reshape(-1)
        rep = items[int(np.argmax(sims_sn))]
        clusters_out.append({
            "cluster": int(c),
            "n_snippets": len(items),
            "representative": rep,
            "snippets": items[:10]
        })

# ================== EXPORT & PRINT ==================
import pandas as pd
pd.DataFrame([{"snippet": s} for s in snippets]).to_csv("snippets_interdisciplina.csv", index=False)
pd.DataFrame([{"concept": w, "freq": f} for w, f in top_concepts]).to_csv("conceptos_interdisciplina.csv", index=False)

summary = {
    "axis": "interdisciplina",
    "axis_terms": AXIS_TERMS,
    "snippets": snippets[:80],
    "concepts": [{"concept": w, "freq": int(f)} for w, f in top_concepts],
    "clusters": clusters_out
}
Path("resumen_interdisciplina.json").write_text(json.dumps(summary, ensure_ascii=False, indent=2), encoding="utf-8")

print("=== Fragmentos (2–8 palabras) relacionados con 'interdisciplina' ===")
for s in snippets[:30]:
    print("-", s)

print("\n=== Conceptos (2–3 palabras) ===")
for w, f in top_concepts[:20]:
    print(f"- {w} ({f})")

if clusters_out:
    print("\n=== Subtemas (clusters sobre fragmentos) ===")
    for cl in sorted(clusters_out, key=lambda d: d["cluster"]):
        print(f"\n[Cluster {cl['cluster']}] n={cl['n_snippets']}")
        print("Representativa:", cl["representative"])
        for s in cl["snippets"]:
            print("-", s)

print("\n✅ Listo. Guardados: 'snippets_interdisciplina.csv', 'conceptos_interdisciplina.csv' y 'resumen_interdisciplina.json'")


=== Fragmentos (2–8 palabras) relacionados con 'interdisciplina' ===
- disciplinas como lo analogico
- el proceso
- bloque estudiantes - interdisciplina josefa quezada buenos
- interdisciplina josefa quezada
- el proyecto
- la experiencia
- el espacio arquitectonico
- la narrativa
- espacio arquitectonico
- este proyecto
- mi proyecto
- esta narrativa racional y lineal
- narrativa racional
- diseno especulativo
- es un espacio abierto
- espacio abierto
- el diseno especulativo
- dia 1 bloque estudiantes - interdisciplina josefa quezada
- la combinacion entre el espacio arquitectonico y tecnologias
- y para hacer una introduccion a este proyecto
- le voy a presentar mi proyecto el cual
- y el corazon del proyecto es la musica
- es por eso que este proyecto invita al
- es un espacio abierto dispuesto a ser habitado
- es por eso que el diseno especulativo lo

=== Conceptos (2–3 palabras) ===
- diseno especulativo (3)
- espacio arquitectonico (2)
- espacio abierto (2)
- disciplina analogic